In [25]:
#!conda install -c conda-forge folium --yes
import folium

In [26]:
import pandas as pd
import numpy as np
import requests
import io
from IPython.display import IFrame

In [27]:
#URL for Baltimore Crime Data
BALT_URL = 'https://data.baltimorecity.gov/api/views/wsfq-mvij/rows.csv?accessType=DOWNLOAD'
s=requests.get(BALT_URL).content
crimedata=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [28]:
crimedata.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,vri_name1,Total Incidents
0,10/12/2019,00:30:00,6G,5900 THEODORE AVE,LARCENY,I,NaN,425.0,NORTHEAST,GLENHAM-BELFORD,-76.549072,39.353730,NaN,ROW/TOWNHOUSE-OCC,NaN,1
1,10/12/2019,00:30:00,4E,5900 THEODORE AVE,COMMON ASSAULT,I,NaN,425.0,NORTHEAST,GLENHAM-BELFORD,-76.549072,39.353730,NaN,ROW/TOWNHOUSE-OCC,NaN,1
2,10/12/2019,14:30:00,7A,1700 ARGONNE DR,AUTO THEFT,O,NaN,421.0,NORTHEAST,MORGAN STATE UNIVERS,-76.587124,39.339956,NaN,STREET,NaN,1
3,10/12/2019,10:30:00,4E,800 N LAKEWOOD AVE,COMMON ASSAULT,I,NaN,323.0,EASTERN,MADISON EASTEND,-76.579593,39.300259,NaN,ROW/TOWNHOUSE-OCC,Eastern 2,1
4,10/12/2019,10:30:00,4E,800 N LAKEWOOD AVE,COMMON ASSAULT,I,NaN,323.0,EASTERN,MADISON EASTEND,-76.579593,39.300259,NaN,ROW/TOWNHOUSE-OCC,Eastern 2,1


In [29]:
#Clean up Crime Data
crimedata['CrimeDateTime'] = crimedata['CrimeDate'].astype(str) + ' ' + crimedata['CrimeTime'].astype(str) # Concat Date and TIme Fields
crimedata = crimedata.drop(['CrimeDate', 'CrimeTime'], axis=1) # Delete the old Date and Time Columns 
crimedata['CrimeDateTime']=pd.to_datetime(crimedata['CrimeDateTime'], errors='coerce')  #Fix the Date type
#Drop all rows prior to 2014 since they are invalid or poorly collected
crimedata = crimedata[crimedata['CrimeDateTime'].dt.year >= 2014]
#Drop all rows that have NaN for eitherr Lat or Long
crimedata = crimedata.dropna(subset=['Longitude', 'Latitude']) 

crimedata.shape

(277483, 15)

In [87]:
crimedata.head(5)

,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,vri_name1,Total Incidents,CrimeDateTime
0,6G,5900 THEODORE AVE,LARCENY,I,NaN,425.0,NORTHEAST,GLENHAM-BELFORD,-76.549072,39.353730,NaN,ROW/TOWNHOUSE-OCC,NaN,1,2019-10-12 00:30:00
1,4E,5900 THEODORE AVE,COMMON ASSAULT,I,NaN,425.0,NORTHEAST,GLENHAM-BELFORD,-76.549072,39.353730,NaN,ROW/TOWNHOUSE-OCC,NaN,1,2019-10-12 00:30:00
2,7A,1700 ARGONNE DR,AUTO THEFT,O,NaN,421.0,NORTHEAST,MORGAN STATE UNIVERS,-76.587124,39.339956,NaN,STREET,NaN,1,2019-10-12 14:30:00
3,4E,800 N LAKEWOOD AVE,COMMON ASSAULT,I,NaN,323.0,EASTERN,MADISON EASTEND,-76.579593,39.300259,NaN,ROW/TOWNHOUSE-OCC,Eastern 2,1,2019-10-12 10:30:00
4,4E,800 N LAKEWOOD AVE,COMMON ASSAULT,I,NaN,323.0,EASTERN,MADISON EASTEND,-76.579593,39.300259,NaN,ROW/TOWNHOUSE-OCC,Eastern 2,1,2019-10-12 10:30:00


In [67]:
crimedata['Description'].groupby([crimedata.Description]).agg('count')

Description
AGG. ASSAULT            29958
ARSON                    1225
AUTO THEFT              24514
BURGLARY                40550
COMMON ASSAULT          45434
HOMICIDE                 1780
LARCENY                 61918
LARCENY FROM AUTO       36915
RAPE                     1734
ROBBERY - CARJACKING     2329
ROBBERY - COMMERCIAL     4951
ROBBERY - RESIDENCE      2792
ROBBERY - STREET        19717
SHOOTING                 3666
Name: Description, dtype: int64

In [ ]:
crimedata['CrimeDateTime'].groupby([crimedata.CrimeDateTime.dt.year]).agg('count')

In [81]:
#Baltimore's Longitude and Latitude
latitude = 39.2904
longitude = -76.6122

In [82]:
# Create the map and visualize the boroughs
city_map = folium.Map(location=[latitude, longitude], zoom_start=12) #

In [83]:
sample = crimedata[(crimedata['Description'] == 'HOMICIDE') & (crimedata['CrimeDateTime'].dt.year == 2018)]

In [84]:
sample.shape

(309, 15)

In [85]:
# add the crimes to the map as small blue circles
for lat, lng, Borough in zip(sample.Latitude, sample.Longitude, sample.District):
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        color='blue',
        popup=Borough,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(city_map)

In [86]:
city_map